Intrinsic Calibration

In [1]:
import cv2
import numpy as np
import glob
  
CHECKERBOARD = (7,10)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# 각 체커보드 이미지에 대한 3D 점 벡터를 저장할 벡터 생성
objpoints = []

# 각 체커보드 이미지에 대한 2D 점 벡터를 저장할 벡터 생성
imgpoints = [] 

# 3D 점의 세계 좌표 정의
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

prev_img_shape = None

# 주어진 디렉터리에 저장된 개별 이미지의 경로 추출
images = sorted(glob.glob('/home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/*.jpg'))
for fname in images:
    print(f'Do {fname}')
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # 체커보드 코너 찾기
    # 이미지에서 원하는 개수의 코너가 발견되면 ret = true
    ret, corners = cv2.findChessboardCorners(gray,
                                            CHECKERBOARD,
                                            cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    # 원하는 개수의 코너가 감지되면,
    # 픽셀 좌표 미세조정 -> 체커보드 이미지 표시
    if ret == True:
        objpoints.append(objp)
        # 주어진 2D 점에 대한 픽셀 좌표 미세조정
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        imgpoints.append(corners2)
        # 코너 그리기 및 표시   ``
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        
        name = fname.split('.jpg')[0]+'_cal.jpg'
        cv2.imwrite(name, img)
        print(f'Save img: {name}')
#     cv2.imshow('img',img)
#     cv2.waitKey(0)
# cv2.destroyAllWindows()
h,w = img.shape[:2] # 480, 640
# 알려진 3D 점(objpoints) 값과 감지된 코너의 해당 픽셀 좌표(imgpoints) 전달, 카메라 캘리브레이션 수행
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

Do /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180700419322.jpg
Save img: /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180700419322_cal.jpg
Do /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180836318573.jpg
Save img: /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180836318573_cal.jpg
Do /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180802218663.jpg
Save img: /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180802218663_cal.jpg
Do /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180939818904.jpg
Save img: /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180939818904_cal.jpg
Do /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180955317344.jpg
Save img: /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180955317344_cal.jpg
Do /home/ircv3/HYU-2024-Embedded/jetracer/dataset/intrinsic/20240611180614619302.jpg
Save img: 

In [3]:
print(ret, mtx, dist, rvecs, tvecs)

0.8305667554518997 [[788.02447052   0.         508.7173462 ]
 [  0.         787.53662632 290.1073598 ]
 [  0.           0.           1.        ]] [[-0.37344988  0.23525292  0.00086942  0.00070701 -0.1178794 ]] (array([[ 0.74488437],
       [ 0.66166253],
       [-2.74288444]]), array([[ 1.22786605],
       [-0.80633796],
       [-1.72509757]]), array([[-0.50877182],
       [-0.32559119],
       [-0.99751179]]), array([[ 0.60458408],
       [-0.22809708],
       [ 2.11769667]]), array([[-0.30064373],
       [ 0.02097632],
       [ 1.98928736]]), array([[-0.01600618],
       [ 0.45899248],
       [-3.06712925]]), array([[-0.08176081],
       [ 0.23627092],
       [ 2.11516332]]), array([[-0.16738113],
       [-0.33976184],
       [ 2.100782  ]]), array([[ 0.15331493],
       [ 0.79563531],
       [-1.28886056]]), array([[-0.04513567],
       [-1.44108677],
       [-2.65094015]]), array([[ 0.9648869 ],
       [ 0.46614537],
       [-1.23507478]]), array([[ 0.27261718],
       [-0.41188721

In [6]:
rvecs

(array([[ 0.74488437],
        [ 0.66166253],
        [-2.74288444]]),
 array([[ 1.22786605],
        [-0.80633796],
        [-1.72509757]]),
 array([[-0.50877182],
        [-0.32559119],
        [-0.99751179]]),
 array([[ 0.60458408],
        [-0.22809708],
        [ 2.11769667]]),
 array([[-0.30064373],
        [ 0.02097632],
        [ 1.98928736]]),
 array([[-0.01600618],
        [ 0.45899248],
        [-3.06712925]]),
 array([[-0.08176081],
        [ 0.23627092],
        [ 2.11516332]]),
 array([[-0.16738113],
        [-0.33976184],
        [ 2.100782  ]]),
 array([[ 0.15331493],
        [ 0.79563531],
        [-1.28886056]]),
 array([[-0.04513567],
        [-1.44108677],
        [-2.65094015]]),
 array([[ 0.9648869 ],
        [ 0.46614537],
        [-1.23507478]]),
 array([[ 0.27261718],
        [-0.41188721],
        [ 3.08950794]]),
 array([[-1.08769372],
        [ 0.21379424],
        [-2.77780917]]),
 array([[-0.61362969],
        [ 0.20035835],
        [-1.27314426]]),
 array

In [9]:
import pickle

intrinsic_dict = {}

intrinsic_dict['I'] = mtx
intrinsic_dict['dist'] = dist

with open('calibration_data.pkl', 'wb') as f:
    pickle.dump(intrinsic_dict, f)

In [8]:
intrinsic_dict

{'I': array([[788.02447052,   0.        , 508.7173462 ],
        [  0.        , 787.53662632, 290.1073598 ],
        [  0.        ,   0.        ,   1.        ]]),
 'dist': array([[-0.37344988,  0.23525292,  0.00086942,  0.00070701, -0.1178794 ]])}

In [22]:
import cv2
import numpy as np
import pickle

# 카메라 캘리브레이션 결과 로드
with open('calibration_data.pkl', 'rb') as f:
    calib_data = pickle.load(f)

mtx = np.array(calib_data['I'])
dist = np.array(calib_data['dist'])

# 왜곡 보정을 위한 함수
def undistort_image(image_path, output_path):
    # 이미지 읽기
    img = cv2.imread(image_path)
    if img is None:
        print(f"이미지를 불러올 수 없습니다: {image_path}")
        return

    # 이미지 크기
    h, w = img.shape[:2]

    # 새로운 카메라 행렬 계산
    newcameramtx, _ = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

    # 왜곡 보정
    # dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
    dst = cv2.undistort(img, mtx, dist, None)

    # 보정된 이미지 저장
    cv2.imwrite(output_path, dst)

    print(f"왜곡 보정된 이미지가 저장되었습니다: {output_path}")

# 예제 이미지 파일 경로
image_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182.jpg'
output_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182_undistorted2.jpg'

# 왜곡 보정 함수 호출
undistort_image(image_path, output_path)


왜곡 보정된 이미지가 저장되었습니다: /home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182_undistorted2.jpg


In [1]:
import cv2

# 마우스 클릭 이벤트 콜백 함수
def click_event(event, x, y, flags, params):
    # 왼쪽 버튼 클릭 이벤트를 감지
    if event == cv2.EVENT_LBUTTONDOWN:
        # 클릭한 위치에 빨간 점 그리기
        cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
        # 클릭한 위치의 좌표 출력
        print(f"클릭한 위치의 좌표: ({x}, {y})")
        # 이미지 업데이트
        cv2.imshow('image', img)

# 이미지 파일 경로
image_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182_undistorted2.jpg'
# image_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182.jpg'

# 이미지 읽기
img = cv2.imread(image_path)
if img is None:
    print(f"이미지를 불러올 수 없습니다: {image_path}")
else:
    # 이미지 창 열기
    cv2.imshow('image', img)

    # 마우스 콜백 함수 설정
    cv2.setMouseCallback('image', click_event)

    # 키보드 입력 대기 (q 키를 누르면 종료)
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 모든 창 닫기
    cv2.destroyAllWindows()


클릭한 위치의 좌표: (165, 536)
클릭한 위치의 좌표: (192, 493)
클릭한 위치의 좌표: (220, 455)
클릭한 위치의 좌표: (241, 426)
클릭한 위치의 좌표: (256, 399)


In [73]:
import cv2

corner_list = []

# 체커보드 코너 찾기 함수
def find_checkerboard_corners(image_path, pattern_size):
    # 이미지 읽기
    img = cv2.imread(image_path)

    # 이미지를 그레이스케일로 변환
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 체커보드 코너 찾기
    ret, corners = cv2.findChessboardCorners(gray,
                                            pattern_size,
                                            cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

    # 코너가 모두 찾아졌을 경우
    if ret:
        # 코너 좌표 정확도를 높이기 위해 사각형 영역을 찾음
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.001)
     
        corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        # 코너를 그리기 위해 이미지에 사각형 그리기
        cv2.drawChessboardCorners(img, pattern_size, corners, ret)

        # 코너 순서 정렬
        corners = corners.reshape(-1, 2)
        sorted_corners = sorted(corners, key=lambda x: x[0])

        # 각 코너의 좌표 출력
        for i, corner in enumerate(sorted_corners):
            print(f"코너 {i+1}: ({corner[0]}, {corner[1]})")
            corner_list.append((corner[0], corner[1]))

        # 이미지 출력
        cv2.imshow('Chessboard Corners', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    else:
        print("체커보드 코너를 찾을 수 없습니다.")

# 이미지 파일 경로
image_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611220230057447.jpg'
# 체커보드의 패턴 사이즈 (가로, 세로)
pattern_size = (5, 7)

# 체커보드 코너 찾기
find_checkerboard_corners(image_path, pattern_size)


체커보드 코너를 찾을 수 없습니다.


In [6]:
sorted_corner_list = []
idx_list = [1, 2, 3, 4, 6, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 30, 29, 28, 27, 26, 35, 34, 33, 32, 31]
for idx in idx_list:
    sorted_corner_list.append(corner_list[idx-1])

In [8]:
import pickle

with open('corner_data.pkl', 'wb') as f:
    pickle.dump(sorted_corner_list, f)

In [103]:
import cv2
import numpy as np
import pickle

# 카메라 캘리브레이션 결과 로드
with open('calibration_data.pkl', 'rb') as f:
    calib_data = pickle.load(f)

mtx = np.array(calib_data['I'])
dist = np.array(calib_data['dist'])

img = cv2.imread('/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182.jpg')

# 이미지의 크기
h, w, _ = img.shape

newcameramtx, _ = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

# 왜곡된 좌표를 보정하는 함수
def undistort_points(points, mtx, dist):
    # 2D 좌표를 numpy 배열로 변환
    points = np.array(points, dtype=np.float32)

    # 왜곡 보정
    undistorted_points = cv2.undistortPoints(points, mtx, dist)

    # 보정된 좌표를 리스트로 변환하여 반환
    undistorted_points = undistorted_points.squeeze().tolist()
    return undistorted_points


with open('corner_data.pkl', 'rb') as f:
    corners_2d= pickle.load(f)
    
# corners_2d = [(0,0), (0, 540), (480, 270), (960, 0), (960, 540)]

# 왜곡 보정된 좌표 얻기
undistorted_corners = undistort_points(corners_2d, mtx, dist)

undistorted_corners2 = []

# 보정된 좌표를 이미지의 픽셀 좌표로 변환
for undistorted_corner in undistorted_corners:
    x = (undistorted_corner[0] + 1) / 2 * w
    y = (undistorted_corner[1] + 1) / 2 * h
    undistorted_corners2.append([x,y])

# 결과 출력
print("보정 전 2D 좌표:", corners_2d)
print(undistorted_corners)
print("보정 후 2D 좌표:", undistorted_corners2)


보정 전 2D 좌표: [(195.41309, 513.9918), (216.20834, 477.70856), (235.72887, 446.57462), (251.86865, 418.84705), (268.28427, 393.80603), (262.82846, 521.4799), (280.5333, 483.68094), (295.16324, 450.68176), (308.51517, 422.16003), (320.7946, 395.35403), (333.93436, 527.4958), (346.9622, 488.2271), (357.17233, 454.17603), (367.46185, 423.29288), (375.9442, 396.48553), (409.39368, 530.25476), (415.83325, 490.58817), (421.82788, 455.25034), (427.6338, 424.43558), (432.1329, 396.44046), (485.53513, 531.6587), (486.6309, 491.56192), (488.32248, 455.90588), (488.425, 425.50977), (489.30695, 398.47726), (562.56616, 531.59186), (557.8272, 491.24854), (553.8702, 457.2212), (549.59607, 426.32224), (546.5076, 399.6125), (638.64056, 529.1526), (628.3758, 490.6888), (618.61505, 456.3289), (610.6416, 426.4589), (603.26666, 399.41315)]
[[-0.4377402365207672, 0.31256139278411865], [-0.40108779072761536, 0.25707343220710754], [-0.36893346905708313, 0.2113451361656189], [-0.34339824318885803, 0.1720371693372

In [105]:
len(undistorted_corners2)

35

In [53]:
import cv2
import numpy as np
import pickle

image_points = np.array(undistorted_corners2)

print(image_points)

# 카메라 캘리브레이션 결과 로드
with open('calibration_data.pkl', 'rb') as f:
    calib_data = pickle.load(f)

mtx = np.array(calib_data['I'])
dist = np.array(calib_data['dist'])

object_points = np.array([
    [12, 8],
    [14, 8],
    [16, 8],
    [20, 8],
    [22, 8],
    [12, 6],
    [14, 6],
    [16, 6],
    [20, 6],
    [22, 6],
    [12, 4],
    [14, 4],
    [16, 4],
    [20, 4],
    [22, 4],
    [12, 2],
    [14, 2],
    [16, 2],
    [20, 2],
    [22, 2],
    [12, 0],
    [14, 0],
    [16, 0],
    [20, 0],
    [22, 0],
    [12, -2],
    [14, -2],
    [16, -2],
    [20, -2],
    [22, -2],
    [12, -4],
    [14, -4],
    [16, -4],
    [20, -4],
    [22, -4]
], dtype=np.float32)

# 카메라 외부 파라미터 추정
retval, rvec, tvec, inliers = cv2.solvePnPRansac(object_points, image_points, mtx, dist)

# 회전 벡터(rvec)와 이동 벡터(tvec)를 회전행렬(R)과 변환 벡터(T)로 변환
R, _ = cv2.Rodrigues(rvec)
T = tvec

# 외부 파라미터 출력
print("Rotation Matrix:\n", R)
print("Translation Vector:\n", T)


[[269.88468647 354.39157605]
 [287.47786045 339.4098267 ]
 [302.91193485 327.06318676]
 [315.16884327 316.45003572]
 [327.09244251 307.07943276]
 [318.92427921 355.2143839 ]
 [332.91955948 340.1597482 ]
 [343.96066189 327.52399698]
 [353.59258175 316.89192429]
 [362.17749596 307.11274981]
 [367.54868746 355.88253558]
 [377.36589432 340.69851816]
 [384.78904009 327.97904581]
 [391.93029642 316.71095163]
 [397.66746998 307.10924953]
 [416.89131975 355.79682291]
 [421.68401599 340.7895577 ]
 [425.92596173 327.81025574]
 [429.8583591  316.73175216]
 [432.85745144 306.82681695]
 [465.32758087 355.81421971]
 [466.18520647 340.78201801]
 [467.35221416 327.78711364]
 [467.49083161 316.9329299 ]
 [468.08540672 307.41480812]
 [514.00573969 355.90206474]
 [510.66846371 340.7402721 ]
 [507.98042715 328.31125483]
 [505.18658817 317.2511901 ]
 [503.19171309 307.83202976]
 [562.75139809 355.71185052]
 [555.293262   341.04698539]
 [548.52456808 328.33969161]
 [543.1418252  317.54451379]
 [538.29032779

error: OpenCV(4.5.4) /home/ubuntu/build_opencv/opencv/modules/calib3d/src/solvepnp.cpp:241: error: (-215:Assertion failed) npoints >= 4 && npoints == std::max(ipoints.checkVector(2, CV_32F), ipoints.checkVector(2, CV_64F)) in function 'solvePnPRansac'


In [87]:
pts2_

[[30, 0],
 [32, 0],
 [34, 0],
 [36, 0],
 [38, 0],
 [40, 0],
 [44, 0],
 [48, 0],
 [52, 0],
 [52, -8],
 [30, -8],
 [32, 8],
 [36, 8],
 [38, 4],
 [32, 2],
 [36, 2],
 [36, -6],
 [44, -2],
 [46, -4],
 [46, 4],
 [50, 6],
 [34, -2],
 [38, -4],
 [40, -6],
 [42, -8],
 [34, 8],
 [34, -6]]

In [93]:
undistorted_corners2+pts1_

[[269.88468647003174, 354.39157605171204],
 [287.47786045074463, 339.40982669591904],
 [302.9119348526001, 327.0631867647171],
 [315.16884326934814, 316.4500357210636],
 [327.0924425125122, 307.0794327557087],
 [318.92427921295166, 355.21438390016556],
 [332.91955947875977, 340.15974819660187],
 [343.96066188812256, 327.5239969789982],
 [353.5925817489624, 316.89192429184914],
 [362.1774959564209, 307.1127498149872],
 [367.54868745803833, 355.8825355768204],
 [377.36589431762695, 340.6985181570053],
 [384.78904008865356, 327.9790458083153],
 [391.930296421051, 316.7109516263008],
 [397.6674699783325, 307.10924953222275],
 [416.8913197517395, 355.79682290554047],
 [421.6840159893036, 340.7895576953888],
 [425.9259617328644, 327.8102557361126],
 [429.85835909843445, 316.7317521572113],
 [432.8574514389038, 306.8268169462681],
 [465.32758086919785, 355.81421971321106],
 [466.1852064728737, 340.78201800584793],
 [467.35221415758133, 327.7871136367321],
 [467.49083161354065, 316.93292990326

In [102]:
len([
    [12, 8],
    [14, 8],
    [16, 8],
    [20, 8],
    [22, 8],
    [12, 6],
    [14, 6],
    [16, 6],
    [20, 6],
    [22, 6],
    [12, 4],
    [14, 4],
    [16, 4],
    [20, 4],
    [22, 4],
    [12, 2],
    [14, 2],
    [16, 2],
    [20, 2],
    [22, 2],
    [12, 0],
    [14, 0],
    [16, 0],
    [20, 0],
    [22, 0],
    [12, -2],
    [14, -2],
    [16, -2],
    [20, -2],
    [22, -2],
    [12, -4],
    [14, -4],
    [16, -4],
    [20, -4],
    [22, -4]
]+pts2_)

62

In [151]:
len(undistorted_corners2)

35

In [161]:
import random
type((random.sample(range(0,35), 20)+list(range(35,71)))[0])

int

In [162]:
import cv2
import numpy as np
import random

# 매칭되는 여러 포인트들 (2D)
pts1 = undistorted_corners2+pts1_
pts2 = [
    [12, 8],
    [14, 8],
    [16, 8],
    [20, 8],
    [22, 8],
    [12, 6],
    [14, 6],
    [16, 6],
    [20, 6],
    [22, 6],
    [12, 4],
    [14, 4],
    [16, 4],
    [20, 4],
    [22, 4],
    [12, 2],
    [14, 2],
    [16, 2],
    [20, 2],
    [22, 2],
    [12, 0],
    [14, 0],
    [16, 0],
    [20, 0],
    [22, 0],
    [12, -2],
    [14, -2],
    [16, -2],
    [20, -2],
    [22, -2],
    [12, -4],
    [14, -4],
    [16, -4],
    [20, -4],
    [22, -4]
]+pts2_
n = 20
ll = random.sample(range(0,35), n)+list(range(35,71))
pts2 = np.array(pts2, dtype=np.float32)[ll]
pts1 = np.array(pts1 , dtype=np.float32)[ll]
print(pts1.shape)
print(pts2.shape)

# 2D에서 2D로 변환하는 행렬 찾기
H, _ = cv2.findHomography(pts1, pts2, cv2.RANSAC)

print("원근 변환 행렬:\n", H)


(56, 2)
(56, 2)
원근 변환 행렬:
 [[ 9.19665261e-03 -4.68996781e-01  1.89779305e+02]
 [-1.20145758e-01 -2.58232567e-02  6.45659908e+01]
 [-6.85361066e-05  4.14751866e-03  1.00000000e+00]]


In [147]:
 
src1 = cv2.imread('IMG_8234', cv2.IMREAD_GRAYSCALE)
src2 = cv2.imread('/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611230616208539.jpg', cv2.IMREAD_GRAYSCALE)

# 특징점 알고리즘 객체 생성 (KAZE, AKAZE, ORB 등)
# feature = cv2.KAZE_create() # 기본값인 L2놈 이용
feature = cv2.AKAZE_create()
# feature = cv2.ORB_create()

# 특징점 검출 및 기술자 계산
kp1, desc1 = feature.detectAndCompute(src1, None)
kp2, desc2 = feature.detectAndCompute(src2, None)

# 특징점 매칭
matcher = cv2.BFMatcher_create()
matches = matcher.match(desc1, desc2)

# 좋은 매칭 결과 선별
matches = sorted(matches, key=lambda x: x.distance)
good_matches = matches[:80]

print('# of kp1:', len(kp1))
print('# of kp2:', len(kp2))
print('# of matches:', len(matches))
print('# of good_matches:', len(good_matches))

# 호모그래피 계산
# DMatch 객체에서 queryIdx와 trainIdx를 받아와서 크기와 타입 변환하기
pts1 = np.array([kp1[m.queryIdx].pt for m in good_matches]
				).reshape(-1, 1, 2).astype(np.float32)

print(pts1.shape)
pts2 = np.array([kp2[m.trainIdx].pt for m in good_matches]
				).reshape(-1, 1, 2).astype(np.float32)
                
H2, _ = cv2.findHomography(pts1, pts2, cv2.RANSAC) # pts1과 pts2의 행렬 주의 (N,1,2)


# 호모그래피를 이용하여 기준 영상 영역 표시
dst = cv2.drawMatches(src1, kp1, src2, kp2, good_matches, None,
                      flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

(h, w) = src1.shape[:2]

# 입력 영상의 모서리 4점 좌표
corners1 = np.array([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]
                    ).reshape(-1, 1, 2).astype(np.float32)

# 입력 영상에 호모그래피 H 행렬로 투시 변환
corners2 = cv2.perspectiveTransform(corners1, H2)

# corners2는 입력 영상에 좌표가 표현되있으므로 입력영상의 넓이 만큼 쉬프트
corners2 = corners2 + np.float32([w, 0])

# 다각형 그리기
cv2.polylines(dst, [np.int32(corners2)], True, (0, 255, 0), 2, cv2.LINE_AA)

cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

error: OpenCV(4.5.4) /home/ubuntu/build_opencv/opencv/modules/features2d/src/akaze.cpp:172: error: (-215:Assertion failed) ! image.empty() in function 'detectAndCompute'


In [137]:
H2

array([[-1.57493973e-01, -8.78507340e-01,  4.45222703e+02],
       [-5.30711965e-02, -2.97262338e-01,  1.50657251e+02],
       [-3.54988791e-04, -1.97259242e-03,  1.00000000e+00]])

In [55]:
import cv2
import numpy as np


# 원본 이미지 상의 점들

# 변환된 점들 계산
pts1_homogeneous = np.concatenate([pts1, np.ones((pts1.shape[0], 1))], axis=1)
pts2_transformed_homogeneous = np.dot(H, pts1_homogeneous.T).T
pts2_transformed = pts2_transformed_homogeneous[:, :2] / pts2_transformed_homogeneous[:, 2:]

# 변환된 점들 시각화
image = np.zeros((1000, 1000, 3), dtype=np.uint8)  # 흰색 배경 이미지 생성
for pt1, pt2, pt3 in zip(pts1, pts2_transformed, pts2):
    print(pt1, pt2, pt3)


[269.88467 354.39157] [11.79848548  7.82283512] [12.  8.]
[287.47787 339.40982] [14.24874993  7.93187799] [14.  8.]
[302.91193 327.0632 ] [16.7313846  7.9971346] [16.  8.]
[315.16885 316.45004] [19.31521435  8.11865059] [20.  8.]
[327.09244 307.07944] [22.05703126  8.18360046] [22.  8.]
[318.9243 355.2144] [11.70614152  5.85818687] [12.  6.]
[332.91956 340.15976] [14.14695148  5.90462363] [14.  6.]
[343.96066 327.524  ] [16.6688353   5.98672111] [16.  6.]
[353.5926  316.89194] [19.24370629  6.05240626] [20.  6.]
[362.1775  307.11276] [22.09837226  6.14122605] [22.  6.]
[367.54868 355.88254] [11.6364898   3.92742227] [12.  4.]
[377.3659  340.69852] [14.0828879   3.94106683] [14.  4.]
[384.78903 327.97903] [16.60754384  3.99387716] [16.  4.]
[391.9303  316.71094] [19.33747268  4.02080525] [20.  4.]
[397.66748 307.10925] [22.15227793  4.06894258] [22.  4.]
[416.89133 355.7968 ] [11.67624297  1.98771383] [12.  2.]
[421.68402 340.78955] [14.09904715  1.99727934] [14.  2.]
[425.92596 327.810

In [52]:
np.dot(M, [[195.41309], [513.9918], [1]])

array([[-1.96322877],
       [-3.62116003],
       [-0.77574418]])

In [45]:
M = cv2.getPerspectiveTransform(pts1[[1, 16, 8, 24], ...], pts2[[1, 16, 8, 24], ...])

In [46]:
M

array([[-6.59711064e-03,  4.63068258e-03, -3.05419987e+00],
       [ 9.90073256e-03, -4.46082673e-03, -3.26306441e+00],
       [-4.33999944e-04, -3.28980913e-03,  1.00000000e+00]])

In [50]:
np.dot(M, np.array([269.88467, 354.39157, 1]).T)

array([-3.19358402, -2.17188786, -0.28301055])

In [44]:
M

array([[ 1.32149891e-01,  1.36366379e-01, -8.73302817e+01],
       [ 1.61139693e-03, -3.00802744e-02,  8.00000000e+00],
       [ 2.01424616e-04, -3.76003430e-03,  1.00000000e+00]])

In [28]:
camera_points = np.dot(R, object_points.T) + T.reshape(3, 1)

# 이미지 좌표계로 변환
image_points_calculated, _ = cv2.projectPoints(camera_points.T, np.zeros((3,)), np.zeros((3,)), mtx, dist)

print("계산된 이미지 좌표:", image_points_calculated[:, 0, :])
print("실제 이미지 좌표:", image_points)

계산된 이미지 좌표: [[194.9570171  515.44361803]
 [216.55078671 478.99210272]
 [235.77043534 447.10654788]
 [268.19426892 394.42428426]
 [281.89879324 372.55438308]
 [262.47829125 521.38399545]
 [279.94117623 483.47356685]
 [295.36986074 450.51059197]
 [321.1743753  396.42722776]
 [332.00156229 374.10558206]
 [334.209224   526.20839833]
 [346.74528716 487.12203591]
 [357.76086181 453.29720362]
 [376.06473883 398.1047955 ]
 [383.70274706 375.42826265]
 [409.12496297 529.652171  ]
 [416.10124269 489.76444991]
 [422.21514249 455.35454519]
 [432.34190991 399.41020853]
 [436.5565082  376.49206876]
 [485.87113118 531.51666537]
 [486.91129805 491.27514864]
 [487.83671232 456.60258145]
 [489.39553693 400.30985645]
 [490.05322359 377.27484881]
 [562.88412323 531.70997174]
 [557.93483093 491.59520861]
 [553.6309397  457.00275351]
 [546.5658727  400.78602633]
 [543.64740312 377.76417183]
 [638.57307286 530.2631115 ]
 [627.91295977 490.74027882]
 [618.59078703 456.56201985]
 [603.18710294 400.8381612 ]
 [

In [25]:
retval, rvec, tvec, inliers

(True,
 array([[ 1.31899468],
        [-1.35577785],
        [ 1.16632694]]),
 array([[-1.33786766],
        [ 4.59736147],
        [21.46669701]]),
 array([[ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 6],
        [ 7],
        [ 8],
        [ 9],
        [11],
        [12],
        [13],
        [14],
        [16],
        [17],
        [18],
        [19],
        [20],
        [21],
        [22],
        [23],
        [24],
        [25],
        [26],
        [27],
        [28],
        [29],
        [30],
        [31],
        [32],
        [33],
        [34]], dtype=int32))

In [22]:
np.dot(R, (12,8,0)) + T.flatten()

array([-8.59214198,  6.16586201, 19.58693257])

In [29]:
import pickle

extrinsic_dict = {}

extrinsic_dict['T'] = T
extrinsic_dict['R'] = R

with open('calibration_data2.pkl', 'wb') as f:
    pickle.dump(extrinsic_dict, f)

In [16]:
np.dot(R, np.array([x2, y2, 1]))

NameError: name 'x2' is not defined

In [165]:
import cv2
import pickle

img = cv2.imread('/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182.jpg')

# 이미지의 크기
h, w, _ = img.shape

with open('calibration_data.pkl', 'rb') as f:
    calib_data = pickle.load(f)

mtx = np.array(calib_data['I'])
dist = np.array(calib_data['dist'])

with open('calibration_data2.pkl', 'rb') as f:
    calib_data2 = pickle.load(f)

T = np.array(calib_data2['T'])
R = np.array(calib_data2['R'])

# 마우스 클릭 이벤트 콜백 함수
def click_event(event, x, y, flags, params):
    
    # 왼쪽 버튼 클릭 이벤트를 감지
    if event == cv2.EVENT_LBUTTONDOWN:
        # 클릭한 위치에 빨간 점 그리기
        cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
        
        distorted_point = np.array([[x, y]], dtype=np.float32)
        undistorted_point = cv2.undistortPoints(distorted_point, mtx, dist)

        x2 = (undistorted_point[0][0][0] + 1) / 2 * w
        y2 = (undistorted_point[0][0][1] + 1) / 2 * h
        
        pts1 = np.array([[x2, y2]], dtype=np.float32)
        
        pts1_homogeneous = np.concatenate([pts1, np.ones((pts1.shape[0], 1))], axis=1)
        pts2_transformed_homogeneous = np.dot(H, pts1_homogeneous.T).T
        pts2_transformed = pts2_transformed_homogeneous[:, :2] / pts2_transformed_homogeneous[:, 2:]
        
        print("입력 x, y 좌표:", x, y)
        print("왜곡 보정된 포인트:", x2, y2)
        print("3D 포인트:", pts2_transformed)
        
        # 이미지 업데이트
        cv2.imshow('image', img)

# 이미지 파일 경로
# image_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182_undistorted2.jpg'
# image_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182.jpg'
image_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611220230057447.jpg'

# 이미지 읽기
img = cv2.imread(image_path)
if img is None:
    print(f"이미지를 불러올 수 없습니다: {image_path}")
else:
    # 이미지 창 열기
    cv2.imshow('image', img)
    # 마우스 콜백 함수 설정
    cv2.setMouseCallback('image', click_event)

    # 키보드 입력 대기 (q 키를 누르면 종료)
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 모든 창 닫기
    cv2.destroyAllWindows()


입력 x, y 좌표: 490 305
왜곡 보정된 포인트: 468.59467685222626 275.10735612362623
3D 포인트: [[30.85230477  0.55107552]]
입력 x, y 좌표: 491 262
왜곡 보정된 포인트: 469.19924318790436 260.35590324550867
3D 포인트: [[35.1561313   0.71810965]]
입력 x, y 좌표: 491 236
왜곡 보정된 포인트: 469.1834905743599 251.40939876437187
3D 포인트: [[37.89177168  0.84720992]]
입력 x, y 좌표: 488 207
왜곡 보정된 포인트: 467.31769323349 241.37151516973972
3D 포인트: [[41.0725941   1.11055481]]


In [2]:
import numpy as np
np.array([1,1,1]).shape

(3,)

In [114]:
import cv2
import pickle

img = cv2.imread('/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182.jpg')

# 이미지의 크기
h, w, _ = img.shape

with open('calibration_data.pkl', 'rb') as f:
    calib_data = pickle.load(f)

mtx = np.array(calib_data['I'])
dist = np.array(calib_data['dist'])

with open('calibration_data2.pkl', 'rb') as f:
    calib_data2 = pickle.load(f)

T = np.array(calib_data2['T'])
R = np.array(calib_data2['R'])

# pts1_ = []
# pts2_ = []

# 마우스 클릭 이벤트 콜백 함수
def click_event(event, x, y, flags, params):
    
    global pts1, pts2
    
    # 왼쪽 버튼 클릭 이벤트를 감지
    if event == cv2.EVENT_LBUTTONDOWN:
        # 클릭한 위치에 빨간 점 그리기
        cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
        
        distorted_point = np.array([[x, y]], dtype=np.float32)
        undistorted_point = cv2.undistortPoints(distorted_point, mtx, dist)

        x2 = (undistorted_point[0][0][0] + 1) / 2 * w
        y2 = (undistorted_point[0][0][1] + 1) / 2 * h
        
        pts1 = np.array([[x2, y2]], dtype=np.float32)
        
        pts1_homogeneous = np.concatenate([pts1, np.ones((pts1.shape[0], 1))], axis=1)
        pts2_transformed_homogeneous = np.dot(H, pts1_homogeneous.T).T
        pts2_transformed = pts2_transformed_homogeneous[:, :2] / pts2_transformed_homogeneous[:, 2:]
        
        print("입력 x, y 좌표:", x, y)
        print("왜곡 보정된 포인트:", x2, y2)
        print("3D 포인트:", pts2_transformed)
        cv2.imshow('image', img)
        
        pts1_.append([x, y])
        pts2_.append(list(map(int, input().split(' '))))
        print(pts1_)
        print(pts2_)
        

# 이미지 파일 경로
# image_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182_undistorted2.jpg'
# image_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611185454173182.jpg'
image_path = '/home/ircv3/HYU-2024-Embedded/jetracer/capture/20240611220230057447.jpg'

# 이미지 읽기
img = cv2.imread(image_path)
if img is None:
    print(f"이미지를 불러올 수 없습니다: {image_path}")
else:
    # 이미지 창 열기
    cv2.imshow('image', img)
    # 마우스 콜백 함수 설정
    cv2.setMouseCallback('image', click_event)

    # 키보드 입력 대기 (q 키를 누르면 종료)
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 모든 창 닫기
    cv2.destroyAllWindows()


입력 x, y 좌표: 463 208
왜곡 보정된 포인트: 451.99116468429565 241.6884557157755
3D 포인트: [[43.37319867  2.11810282]]
[[492, 307], [491, 293], [490, 283], [490, 271], [490, 261], [490, 252], [491, 236], [491, 221], [491, 208], [599, 211], [660, 309], [331, 292], [344, 267], [419, 260], [449, 293], [453, 271], [602, 272], [522, 236], [550, 229], [430, 227], [406, 215], [529, 283], [561, 262], [592, 252], [618, 243], [366, 233], [584, 237], [463, 208]]
[[30, 0], [32, 0], [34, 0], [36, 0], [38, 0], [40, 0], [44, 0], [48, 0], [52, 0], [52, -8], [30, -8], [32, 8], [36, 8], [38, 4], [32, 2], [36, 2], [36, -6], [44, -2], [46, -4], [46, 4], [50, 6], [34, -2], [38, -4], [40, -6], [42, -8], [34, 8], [34, -6], [52, 2]]
입력 x, y 좌표: 436 208
왜곡 보정된 포인트: 435.36165833473206 241.6306507587433
3D 포인트: [[43.8457738   3.26411258]]
[[492, 307], [491, 293], [490, 283], [490, 271], [490, 261], [490, 252], [491, 236], [491, 221], [491, 208], [599, 211], [660, 309], [331, 292], [344, 267], [419, 260], [449, 293], [453, 271

In [82]:
pts1_

[[492, 307],
 [491, 293],
 [490, 283],
 [490, 271],
 [490, 261],
 [490, 252],
 [491, 236],
 [491, 221],
 [491, 208],
 [599, 211],
 [660, 309],
 [331, 292],
 [344, 267],
 [419, 260],
 [449, 293],
 [453, 271],
 [602, 272],
 [522, 236],
 [550, 229],
 [430, 227],
 [406, 215],
 [529, 283],
 [561, 262],
 [592, 252],
 [618, 243]]

In [80]:
pts2_

[[30, 0],
 [32, 0],
 [34, 0],
 [36, 0],
 [38, 0],
 [40, 0],
 [44, 0],
 [48, 0],
 [52, 0],
 [52, -8],
 [30, -8],
 [32, 8],
 [36, 8],
 [38, 4],
 [32, 2],
 [36, 2],
 [36, -6],
 [44, -2],
 [46, -4],
 [46, 4],
 [50, 6],
 [34, -2],
 [38, -4],
 [40, -6],
 [42, -8]]

In [148]:
pts1_

[[492, 307],
 [491, 293],
 [490, 283],
 [490, 271],
 [490, 261],
 [490, 252],
 [491, 236],
 [491, 221],
 [491, 208],
 [599, 211],
 [660, 309],
 [331, 292],
 [344, 267],
 [419, 260],
 [449, 293],
 [453, 271],
 [602, 272],
 [522, 236],
 [550, 229],
 [430, 227],
 [406, 215],
 [529, 283],
 [561, 262],
 [592, 252],
 [618, 243],
 [366, 233],
 [584, 237],
 [463, 208],
 [436, 208],
 [407, 215],
 [375, 221],
 [491, 215],
 [518, 209],
 [548, 216],
 [578, 222],
 [599, 211]]

In [149]:
import pickle

homo_dict = {}

homo_dict['src'] = pts1_
homo_dict['dst'] = pts2_

with open('homography_data.pkl', 'wb') as f:
    pickle.dump(homo_dict, f)

In [150]:
homo_dict

{'src': [[492, 307],
  [491, 293],
  [490, 283],
  [490, 271],
  [490, 261],
  [490, 252],
  [491, 236],
  [491, 221],
  [491, 208],
  [599, 211],
  [660, 309],
  [331, 292],
  [344, 267],
  [419, 260],
  [449, 293],
  [453, 271],
  [602, 272],
  [522, 236],
  [550, 229],
  [430, 227],
  [406, 215],
  [529, 283],
  [561, 262],
  [592, 252],
  [618, 243],
  [366, 233],
  [584, 237],
  [463, 208],
  [436, 208],
  [407, 215],
  [375, 221],
  [491, 215],
  [518, 209],
  [548, 216],
  [578, 222],
  [599, 211]],
 'dst': [[30, 0],
  [32, 0],
  [34, 0],
  [36, 0],
  [38, 0],
  [40, 0],
  [44, 0],
  [48, 0],
  [52, 0],
  [52, -8],
  [30, -8],
  [32, 8],
  [36, 8],
  [38, 4],
  [32, 2],
  [36, 2],
  [36, -6],
  [44, -2],
  [46, -4],
  [46, 4],
  [50, 6],
  [34, -2],
  [38, -4],
  [40, -6],
  [42, -8],
  [34, 8],
  [34, -6],
  [52, 2],
  [52, 4],
  [50, 6],
  [48, 8],
  [50, 0],
  [52, -2],
  [50, -4],
  [48, -6],
  [52, -8]]}